# Lab One: Exploring Table Data

#### By: Harrison Noble and Henry Lambson 

## 1. Business Understanding

According to UCI [Ref 1], this dataset contains "red and white wine varients of the Portuguese Vinho Verde wine". Two datasets were collected, one for the reds and one for the whites.

The data was gathered via physical/chemical tests (such as acid content) and judgements of the quality. The value for the quality was determined by having three wine testers try each wine and score it from 0 to 10, with 0 being very bad and 10 being excellent. The median of the three scores was used in the final dataset.

The wine quality dataset contains 1599 red wine samples and 4898 white wine samples, with 6497 samples in total. There are 12 unique features including the quality value. Each feature is aside from the quality is numerical, with the quality being categorical.      

Through analyzing this dataset, we will be able to help predict which wines should continue to be tested in order to eventually reach the market. If we were to design the algorithm for high quality wine testers, we would only want to send them the samples that are deemed to be a 7 or higher on the quality by the algorithm. In order to accomplish this, we will want to reduce the amount of false-negatives in order to ensure that no top quality wine is passed up on. To this end, some underperforming wines will pass through as false-positives. We accept this tradeoff so long that the algorithm narrows down the samples by 60%, assuming the majority of the wines are rated below 7. This could be useful for wine testers as it cuts down the total amount of samples that would need to be tested.

Additionally, this dataset could be used to predict how the chemical or physical compenents of the wine correspond to the quality. By exploring the relationship between the features provided and the quality of the wine, we will be able to determine which feature values or ratios correspond to higher qualities. Wine producers would be interested in this result as they could determine which features and how much of them lead to higher quality wines. For this model to work well, we would like to limit the false positive rate so that we ensure that the producers are not given poor predictions.


Dataset: Wine Quality URL: https://archive.ics.uci.edu/ml/datasets/wine+quality

## 2. Data Understanding

### 2.1 Data Description

In [66]:
import pandas as pd
import numpy as np

# load red wine dataset into pandas and give it a color attribute (R)
df_red = pd.read_csv('./winequality-red.csv', sep=';')
df_red['color'] = 0

# load white wine dataset into pandas and give it a color attribute (W)
df_white = pd.read_csv('./winequality-white.csv', sep=';')
df_white['color'] = 1

# combine dataset into one pandas dataframe
df_full = pd.concat([df_red, df_white], axis=0, ignore_index=True)

#print number of rows in each df to confirm all data is loaded in
print('Length of red wine dataset:', df_red.shape[0])
print('Length of white wine dataset:', df_white.shape[0])
print('Length of all wine dataset:', df_full.shape[0])

df_full.head()

Length of red wine dataset: 1599
Length of white wine dataset: 4898
Length of all wine dataset: 6497


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0


Although the quality attribute is a number, we decided to consider it as categorical because the number maps to a description of the wine quality (ex: 5 is average). Additionally, we added the color attribute to distinguish white wine (0) from red wine (1) so that we could combine the two datasets into one dataframe.

In [67]:
print(df_full.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  color                 6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 660.0 KB
None


Based on the dataframe info function, we can see that there are no missing values as there are 6497 non-null values for each attribute.

To further confirm this, we decided to run the describe function on the dataset to output statistical information regarding the dataframe. If there was obscure data within this dataset, it would become apparent in the mean, min, or max values for each feature.

In [69]:
#summarize data
df_full.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378,0.753886
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255,0.430779
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000,0.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000,1.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000,1.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000,1.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000,1.000000


As we can see, this table shows no outlandish values. One thing we decided to investigate was the max values for "free sulfur dioxide" and "total sulfur dioxide" as they are many standard deviations away from the mean. In manually checking the dataset, both max values appeared in the same instance, therefore we consider this instance to be an outlier. 

In [71]:
#data description dataframe, idea taken from reference 4
description_df = pd.DataFrame()

description_df['Attributes'] = df_full.columns
#Description of each attribute taken from reference 2
description_df['Description'] = ['grams of tartaric acid per liter', 
                                 'grams of acetic acid per liter',
                                 'grams of citric acid per liter',
                                 'grams of sugar per liter remaining after fermentation stops',
                                 'grams of sodium chloride (salt) per liter',
                                 'milligrams of free form sulfur dioxide per liter',
                                 'milligrams of free form and bound forms of sulfur dioxide per liter',
                                 'density of the wine (grams per cubic centimeter)',
                                 'the pH value of the wine',
                                 'grams of potassium sulphate per liter',
                                 'percent of alcohol by volume',
                                 'median score given by wine tasters',
                                 'type/color of wine']
description_df['Scales'] = ['ratio', 'ratio', 'ratio', 'ratio', 
                            'ratio', 'ratio', 'ratio', 'ratio', 
                            'interval', 'ratio', 'ratio', 'ordinal', 'nominal']
description_df['Discrete\Continuous'] = ['Continuous', 'Continuous', 'Continuous',
                                        'Continuous', 'Continuous', 'Continuous',
                                        'Continuous', 'Continuous', 'Continuous',
                                        'Continuous', 'Continuous', 'Discrete',
                                        'Discrete']
description_df['Range'] = ['3.8 - 15.9', '0.08 - 1.58', '0.0 - 1.66', '0.6 - 65.8',
                            '0.009 - 0.611', '1.0 - 289.0', '6.0 - 440.0',
                            '0.98711 - 1.03898', '2.72 - 4.01', '0.22 - 2.0',
                            '8.0 - 14.9', '3 - 9', '0: red; 1: white']

description_df

,Attributes,Description,Scales,Discrete\Continuous,Range
0,fixed acidity,grams of tartaric acid per liter,ratio,Continuous,3.8 - 15.9
1,volatile acidity,grams of acetic acid per liter,ratio,Continuous,0.08 - 1.58
2,citric acid,grams of citric acid per liter,ratio,Continuous,0.0 - 1.66
3,residual sugar,grams of sugar per liter remaining after ferme...,ratio,Continuous,0.6 - 65.8
4,chlorides,grams of sodium chloride (salt) per liter,ratio,Continuous,0.009 - 0.611
5,free sulfur dioxide,milligrams of free form sulfur dioxide per liter,ratio,Continuous,1.0 - 289.0
6,total sulfur dioxide,milligrams of free form and bound forms of sul...,ratio,Continuous,6.0 - 440.0
7,density,density of the wine (grams per cubic centimeter),ratio,Continuous,0.98711 - 1.03898
8,pH,the pH value of the wine,interval,Continuous,2.72 - 4.01
9,sulphates,grams of potassium sulphate per liter,ratio,Continuous,0.22 - 2.0


The above table gives a description of each feature, type of data (scales), whether it is discrete or continuous, and the max and min value (range). 

Each continuous feature will be represented by a float and each discrete value will be represented by an integer. 

### 2.2 Data Quality

- 1.5 points: Give justifications for your methods (elimination or imputation).  

As shown above, we have no missing data in our dataset, meaning no imputation is necessary. If there were any missing values we would use mean or median imputation for the numerical features and throw out any instances that were missing a quality value. 

Let's now explore if there are duplicate instances or any extreme outliers. 

In [85]:
#find all duplicated instances
duplicated_indexes = df_full.duplicated()
print('Duplicated instances:', len(df_full[duplicated_indexes]))

# pd.set_option("display.max_rows", None, "display.max_columns", None)
# duplicated_whites = df_white.duplicated()
# df_white[duplicated_whites].sort_values(by=['fixed acidity', 'volatile acidity'])

# duplicated_reds = df_red.duplicated()
# df_red[duplicated_reds].sort_values(by=['fixed acidity', 'volatile acidity'])

Duplicated instances: 1177


Given this output we can see that pandas calculated 1177 potential duplicated instances. Through our manual inspection of alleged duplicates (commented out lines), we decided to keep these in the dataset. While we did see there were instances that appeared to be duplicated, the vast majority of the instances in the duplicated_reds and duplicated_whites datasets did not seem to be actual duplicates. From our understanding, the "sort_values" function would display the duplicates next to each other in the table. In our case, we saw this for very few instances, leading us to keep all the data. 

Additionally, we have no way of verifying if the instances that seemed to be duplicates were actually duplicated because this dataset does not contain names or ID values for each wine. 

## 3. Data Visualization

- 2 points: Visualize basic feature distributions. That is, plot the dynamic range and exploratory distribution plots (like boxplots, histograms, kernel density estimation) to better understand the data. Describe anything meaningful or potentially useful you discover from these visualizations. These may also help to understand what data is missing or needs imputation. __Note:__ You can also use data from other sources to bolster visualizations. Visualize at least five plots, at least one categorical and at least one numeric. 
- 2.5 points: Ask three interesting questions that are relevant to your dataset and explore visuals that help answer these questions. Use whichever visualization method is appropriate for your data.  __Important:__ Interpret the implications for each visualization. 

## 4. Exceptional Work (Dimensionality Reduction With PCA)

- You have free reign to provide any additional analyses. 
- One idea (required for 7000 level students): implement dimensionality reduction using uniform manifold approximation and projection (UMAP), then visualize and interpret the results. Give an explanation of UMAP dimensionality reduction methods. You may be interested in the following information: https://github.com/lmcinnes/umap 

### Reference

[1] UCI Machine Learning Repository. Wine Quality Dataset. https://archive.ics.uci.edu/ml/datasets/Wine+Quality

[2] https://rstudio-pubs-static.s3.amazonaws.com/57835_c4ace81da9dc45438ad0c286bcbb4224.html

[3] Penn State Eberly College of Science. Analysis of Wine Quality Data. https://online.stat.psu.edu/stat508/lesson/analysis-wine-quality-data

[4] Graded Example. https://www.dropbox.com/sh/yinn3v3qnzlbep5/AAA-i1u-4ylyxwiCb8nyEDY7a?dl=0&preview=ExampleRendered.html
